**Assignment 4: Adverserial Search (Given: 28 Feb 2023, Due: 19 Mar 2023)**


**General instructions**

* Solutions are to be typed in the `.ipynb` file provided and uploaded in the lab course page in Moodle on the due date. 
* Your code should be well commented and should be compatible with python3.
* For this assignment, you are allowed to import the libraries `random` and `copy` of python3. No other libraries may be imported.


# Adverserial search

The Tic-Tac-Toe game starts on a 3x3 grid with two players "X" and "O" who take turns and play. The rules are as follows: each player gets a turn with player "X" (resp. "O") writing an "X" (resp. "O") in an empty cell of the grid. The game starts with the move of the "O" player. The first player to write on three horizontal or vertical or diagonal cells wins.

(a) Use the minimax strategy to design an AI that plays the game optimally. The leaf nodes where "X" wins gets 1 and "O" wins gets -1 and neither wins gets a zero. A sample game play is given below.


This is a competetive environment in which two or more agents have conflicting goals giving rise to adversial search problems.

Deterministic
Two Player (multi-agent)
Perfect Information, fully observable
Zero-Sum games (only one player can win)
Number of distinct states: 5478
Here the 'X' is not taking risks because it is assuming that the opponenet is playing optimally.

The minmax algorithm performs a complete depth-first search exploration of the Game Tree
If the maximum depth of the Tree is m and there are b legal moves at each point.
Time complexity: O(b^(m)) and space complexity is O(bm) [in general]


In [1]:
class TicTacToeGame:
    '''This class is regarding Tic Tac Toe Board Game'''
    def __init__(self):
        self.grid = [[" " for i in range(3)] for j in range(3)]
        
    def utility(self, player, count):
        if player == "X": # "x" won
            return 1
        elif player == "O": # "x" lost
            return -1
        elif count == 6: # draw match
            return 0
        
    def IsTerminal(self, state):
        count = 0
        # checking all rows
        for i in range(3):
            if state[i][0] != " " and state[i][1] != " " and state[i][2] != " ":
                count += 1
                if state[i][0] == state[i][1] and state[i][0] == state[i][2]:
                    return True, self.utility(state[i][0], count)
        
        # checking all columns
        for j in range(3):
            if state[0][j] != " " and state[1][j] != " " and state[2][j] != " ":
                count += 1
                if state[0][j] == state[1][j] and state[0][j] == state[2][j]:
                    return True, self.utility(state[0][j], count)

        # checking diagonals
        if state[0][0] != " " and state[0][0] == state[1][1] and state[0][0] == state[2][2]:
            return True, self.utility(state[0][0], count)
        if state[0][2] != " " and state[0][2] == state[1][1] and state[0][2] == state[2][0]:
            return True, self.utility(state[0][2], count)
        
        if count == 6:
            return True, self.utility(None, count)
        return False, None
    
    def next_states(self, state, player):
        states = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == " ":
                    temp = []
                    for row in state:
                        temp.append(row.copy())
                    temp[i][j] = player
                    states.append(temp)
        return states
    
    def min_max_search(self, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(self.grid)
        if isterminal is True:
            leaves[0] += 1
            depth[0] = 1
            return
        
        value, state, final_depth = self.max_value(self.grid, 1, leaves)
        depth[0] = final_depth
        for i in range(3):
            for j in range(3):
                self.grid[i][j] = state[i][j]
        return 
    
    def max_value(self, state, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_max = float("-inf")
        assumed_state = None
        states = self.next_states(state, "X")
        
        final_depth = float("-inf")
        assumed_depth = float("inf")
        for s in states:
            temp_max, temp_state, temp_depth = self.min_value(s, depth, leaves)
            final_depth = max(final_depth, temp_depth)
            if temp_max > assumed_max:
                assumed_max = temp_max
                assumed_state = s
                assumed_depth = temp_depth
            elif temp_max == assumed_max and assumed_depth > temp_depth: # chossing stateswith minimum depth
                assumed_state = s
                assumed_depth = temp_depth
        return assumed_max, assumed_state, depth + final_depth
    

    def min_value(self, state, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_min = float("inf")
        assumed_state = None
        states = self.next_states(state, "O")
        
        assumed_depth = float("inf")
        final_depth = float("-inf")
        for s in states:
            temp_min, temp_state, temp_depth = self.max_value(s, depth, leaves)
            final_depth = max(final_depth, temp_depth)
            if temp_min < assumed_min:
                assumed_min = temp_min
                assumed_state = s
                assumed_depth = temp_depth
            elif temp_min == assumed_min and assumed_depth > temp_depth:
                assumed_state = s
                assumed_depth = temp_depth
        return assumed_min, assumed_state, depth + final_depth

In [3]:
obj_game = TicTacToeGame() # Class instantiation
for i in range(3):
    for j in range(3):
        print(3 * i + j + 1, end = "")
        if j == 0 or j == 1:
            print("|", end = "")
    print()
    if i != 2:
        print("-+-+-")
print()

final_depth, final_leaves = 0, 0

k = 0
while 1:
    depth = [0, ]
    leaves = [0, ]
    
    isterminal, utility = obj_game.IsTerminal(obj_game.grid)
    if isterminal is True:
        if utility == 1:
            print("Player X won, utility = 1")
        elif utility == -1:
            print("Player O won, utility = -1")
        else:
            print("Draw")
        break
    
    O_pos = int(input("Enter position for O Player:"))
    
    # The following code gives the present state
    for i in range(3):
        for j in range(3):
            if 3 * i + j + 1 == O_pos:
                obj_game.grid[i][j] = "O"
                
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    
    obj_game.min_max_search(depth, leaves)
    if k == 0:
        final_depth = depth[0]
        final_leaves = leaves[0]
        
    
    for i in range(3):
        for j in range(3):
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    k += 1

1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

Enter position for O Player:5
 | | 
-+-+-
 |O| 
-+-+-
 | | 

X| | 
-+-+-
 |O| 
-+-+-
 | | 

Enter position for O Player:7
X| | 
-+-+-
 |O| 
-+-+-
O| | 

X| |X
-+-+-
 |O| 
-+-+-
O| | 

Enter position for O Player:4
X| |X
-+-+-
O|O| 
-+-+-
O| | 

X|X|X
-+-+-
O|O| 
-+-+-
O| | 

Player X won, utility = 1


In [4]:
class TicTacToeGame1:
    '''This class is regarding Tic Tac Toe Board Game'''
    def __init__(self):
        self.grid = [[" " for i in range(3)] for j in range(3)]
        
    def utility(self, player, count):
        if player == "X": # "x" won
            return 1
        elif player == "O": # "x" lost
            return -1
        elif count == 6: # draw match
            return 0
        
    def IsTerminal(self, state):
        count = 0
        # checking all rows
        for i in range(3):
            if state[i][0] != " " and state[i][1] != " " and state[i][2] != " ":
                count += 1
                if state[i][0] == state[i][1] and state[i][0] == state[i][2]:
                    return True, self.utility(state[i][0], count)
        
        # checking all columns
        for j in range(3):
            if state[0][j] != " " and state[1][j] != " " and state[2][j] != " ":
                count += 1
                if state[0][j] == state[1][j] and state[0][j] == state[2][j]:
                    return True, self.utility(state[0][j], count)

        # checking diagonals
        if state[0][0] != " " and state[0][0] == state[1][1] and state[0][0] == state[2][2]:
            return True, self.utility(state[0][0], count)
        if state[0][2] != " " and state[0][2] == state[1][1] and state[0][2] == state[2][0]:
            return True, self.utility(state[0][2], count)
        
        if count == 6:
            return True, self.utility(None, count)
        return False, None
    
    def next_states(self, state, player):
        states = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == " ":
                    temp = []
                    for row in state:
                        temp.append(row.copy())
                    temp[i][j] = player
                    states.append(temp)
        return states
    
    def min_max_search(self, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(self.grid)
        if isterminal is True:
            leaves[0] += 1
            depth[0] = 1
            return
        
        value, state, final_depth = self.max_value(self.grid, 1, leaves)
        depth[0] = final_depth
        for i in range(3):
            for j in range(3):
                self.grid[i][j] = state[i][j]
        return 
    
    def max_value(self, state, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_max = float("-inf")
        assumed_state = None
        states = self.next_states(state, "X")
        
        final_depth = float("-inf")
        assumed_depth = float("inf")
        for s in states:
            temp_max, temp_state, temp_depth = self.min_value(s, depth, leaves)
            final_depth = max(final_depth, temp_depth)
            if temp_max > assumed_max:
                assumed_max = temp_max
                assumed_state = s
                assumed_depth = temp_depth
            elif temp_max == assumed_max and assumed_depth > temp_depth: # chossing stateswith minimum depth
                # assumed_state = s
                assumed_depth = temp_depth
        return assumed_max, assumed_state, depth + final_depth
    

    def min_value(self, state, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_min = float("inf")
        assumed_state = None
        states = self.next_states(state, "O")
        
        assumed_depth = float("inf")
        final_depth = float("-inf")
        for s in states:
            temp_min, temp_state, temp_depth = self.max_value(s, depth, leaves)
            final_depth = max(final_depth, temp_depth)
            if temp_min < assumed_min:
                assumed_min = temp_min
                assumed_state = s
                assumed_depth = temp_depth
            elif temp_min == assumed_min and assumed_depth > temp_depth:
                # assumed_state = s
                assumed_depth = temp_depth
        return assumed_min, assumed_state, depth + final_depth

In [10]:
obj_game = TicTacToeGame1() # Class instantiation
for i in range(3):
    for j in range(3):
        print(3 * i + j + 1, end = "")
        if j == 0 or j == 1:
            print("|", end = "")
    print()
    if i != 2:
        print("-+-+-")
print()

final_depth, final_leaves = 0, 0

k = 0
while 1:
    depth = [0, ]
    leaves = [0, ]
    
    isterminal, utility = obj_game.IsTerminal(obj_game.grid)
    if isterminal is True:
        if utility == 1:
            print("Player X won, utility = 1")
        elif utility == -1:
            print("Player O won, utility = -1")
        else:
            print("Draw")
        break
    
    O_pos = int(input("Enter position for O Player:"))
    
    # The following code gives the present state
    for i in range(3):
        for j in range(3):
            if 3 * i + j + 1 == O_pos:
                obj_game.grid[i][j] = "O"
                
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    
    obj_game.min_max_search(depth, leaves)
    if k == 0:
        final_depth = depth[0]
        final_leaves = leaves[0]
        
    
    for i in range(3):
        for j in range(3):
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    k += 1

1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

Enter position for O Player:3
 | |O
-+-+-
 | | 
-+-+-
 | | 

 | |O
-+-+-
 |X| 
-+-+-
 | | 

Enter position for O Player:1
O| |O
-+-+-
 |X| 
-+-+-
 | | 

O|X|O
-+-+-
 |X| 
-+-+-
 | | 

Enter position for O Player:7
O|X|O
-+-+-
 |X| 
-+-+-
O| | 

O|X|O
-+-+-
X|X| 
-+-+-
O| | 

Enter position for O Player:8
O|X|O
-+-+-
X|X| 
-+-+-
O|O| 

O|X|O
-+-+-
X|X|X
-+-+-
O|O| 

Player X won, utility = 1


(b) Modify the previous answer to calculate in each game the following:

* the maximum depth of exploration of the game tree in a game, and
* number of leaves of the game tree whose scores were computed be the end of the game.




In [164]:
print("Maximum depth:", final_depth)
print("Total Number of Leaves:", final_leaves)
final_leaves2 = final_leaves

Maximum depth: 9
Total Number of Leaves: 29592


(c) Implement alpha-beta pruning minimax search to solve the Tic-Tac-Toe and repeat part (b). 

Compare your results with vanilla minimax search and see on which all parameters part (b) is alpha-bet pruning search better. Also obtain by what factor is it better.

In [153]:
class TicTacToeGame1:
    '''This class is regarding Tic Tac Toe Board Game'''
    def __init__(self):
        self.grid = [[" " for i in range(3)] for j in range(3)]
        
    def utility(self, player, count):
        if player == "X": # "x" won
            return 1
        elif player == "O": # "x" lost
            return -1
        elif count == 6: # draw match
            return 0
        
    def IsTerminal(self, state):
        count = 0
        # checking all rows
        for i in range(3):
            if state[i][0] != " " and state[i][1] != " " and state[i][2] != " ":
                count += 1
                if state[i][0] == state[i][1] and state[i][0] == state[i][2]:
                    return True, self.utility(state[i][0], count)
        
        # checking all columns
        for j in range(3):
            if state[0][j] != " " and state[1][j] != " " and state[2][j] != " ":
                count += 1
                if state[0][j] == state[1][j] and state[0][j] == state[2][j]:
                    return True, self.utility(state[0][j], count)

        # checking diagonals
        if state[0][0] != " " and state[0][0] == state[1][1] and state[0][0] == state[2][2]:
            return True, self.utility(state[0][0], count)
        if state[0][2] != " " and state[0][2] == state[1][1] and state[0][2] == state[2][0]:
            return True, self.utility(state[0][2], count)
        
        if count == 6:
            return True, self.utility(None, count)
        return False, None
    
    def next_states(self, state, player):
        states = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == " ":
                    temp = []
                    for row in state:
                        temp.append(row.copy())
                    temp[i][j] = player
                    states.append(temp)
        return states
    
    def alpha_beta_search(self, depth, leaves):
        isterminal, assumed_utility = self.IsTerminal(self.grid)
        if isterminal is True:
            leaves[0] += 1
            depth[0] = 1
            return
        
        value, state, final_depth = self.max_value(self.grid, 1, leaves, float("-inf"), float("+inf"))
        depth[0] = final_depth
        for i in range(3):
            for j in range(3):
                self.grid[i][j] = state[i][j]
        return 
    
    def max_value(self, state, depth, leaves, alpha, bheta):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_max_utility = float("-inf")
        assumed_state = None
        states = self.next_states(state, "X")
        
        final_depth = float("-inf")
        assumed_depth = float("inf")
        for s in states:
            temp_max_utility, temp_state, temp_depth = self.min_value(s, depth, leaves, alpha, bheta)
            final_depth = max(final_depth, temp_depth)
            
            if temp_max_utility > assumed_max_utility:
                assumed_max_utility = temp_max_utility
                assumed_state = s
                assumed_depth = temp_depth
                alpha = max(alpha, assumed_max_utility)
            elif temp_max_utility == assumed_max_utility and assumed_depth > temp_depth:
                assumed_depth = temp_depth
                assumed_state = s
            if temp_max_utility >= bheta:
                return assumed_max_utility, assumed_state, depth + final_depth
            
        return assumed_max_utility, assumed_state, depth + final_depth
    

    def min_value(self, state, depth, leaves, alpha, bheta):
        isterminal, assumed_utility = self.IsTerminal(state)
        if isterminal is True:
            leaves[0] += 1
            return assumed_utility, state, 1
        
        assumed_min_utility = float("inf")
        assumed_state = None
        states = self.next_states(state, "O")
        
        assumed_depth = float("inf")
        final_depth = float("-inf")
        for s in states:
            temp_min_utility, temp_state, temp_depth = self.max_value(s, depth, leaves, alpha, bheta)
            final_depth = max(final_depth, temp_depth)
            if temp_min_utility < assumed_min_utility:
                assumed_min_utility = temp_min_utility
                assumed_state = s
                assumed_depth = temp_depth
                bheta = min(bheta, assumed_min_utility)
            elif temp_min_utility == assumed_min_utility and temp_depth < assumed_depth:
                assumed_state = s
                assumed_depth = temp_depth
            if assumed_min_utility <= alpha:
                return assumed_min_utility, assumed_state, depth + final_depth
        return assumed_min_utility, assumed_state, depth + final_depth

In [158]:
obj_game = TicTacToeGame1() # Class instantiation
for i in range(3):
    for j in range(3):
        print(3 * i + j + 1, end = "")
        if j == 0 or j == 1:
            print("|", end = "")
    print()
    if i != 2:
        print("-+-+-")
print()

final_depth, final_leaves = 0, 0

k = 0
while 1:
    depth = [0, ]
    leaves = [0, ]
    
    isterminal, utility = obj_game.IsTerminal(obj_game.grid)
    if isterminal is True:
        if utility == 1:
            print("Player X won, utility = 1")
        elif utility == -1:
            print("Player O won, utility = -1")
        else:
            print("Draw")
        break
    
    O_pos = int(input("Enter position for O Player:"))
    
    # The following code gives the present state
    for i in range(3):
        for j in range(3):
            if 3 * i + j + 1 == O_pos:
                obj_game.grid[i][j] = "O"
                
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    
    obj_game.alpha_beta_search(depth, leaves)
    if k == 0:
        final_depth1 = depth[0]
        final_leaves1 = leaves[0]
        
    
    for i in range(3):
        for j in range(3):
            print(obj_game.grid[i][j], end = "")
            if j == 0 or j == 1:
                print("|", end = "")
        print()
        if i != 2:
            print("-+-+-")
    print()
    k += 1

1|2|3
-+-+-
4|5|6
-+-+-
7|8|9

Enter position for O Player:1
O| | 
-+-+-
 | | 
-+-+-
 | | 

O| | 
-+-+-
 |X| 
-+-+-
 | | 

Enter position for O Player:9
O| | 
-+-+-
 |X| 
-+-+-
 | |O

O|X| 
-+-+-
 |X| 
-+-+-
 | |O

Enter position for O Player:4
O|X| 
-+-+-
O|X| 
-+-+-
 | |O

O|X| 
-+-+-
O|X| 
-+-+-
 |X|O

Player X won, utility = 1


In [165]:
print("Maximum depth:", final_depth1)
print("Total Number of Leaves:", final_leaves1)

Maximum depth: 9
Total Number of Leaves: 929


# Features of program:
a) No need to construct 1st step of choosing position of "O" in Game Tree.
b) State of same utility, then state with minimum distance to terminal nodes is choosed.

In [ ]:
if final_leaves1 < final_leaves2:
    print("On parameter leaves nodes alpha beta search is better than Vanilla search")
    print("Alpha beta search has leaves nodes: ", final_leaves1)
    print("Vanilla Minmax search has leaves nodes: ", final_leaves2)
    print("Approximately alpha beta search is better than Vanilla seach by a factor of", (final_leaves2 - final_leaves1) / final_leaves2)